In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import lightgbm as lgbm
import gc
import xgboost as xgb
import pickle as pickle


from catboost import CatBoostRegressor
import dask.dataframe as dd
from sklearn.model_selection import KFold
from itertools import product

In [192]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [193]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
sales_train = sales_train[sales_train['year'] != 2013]
#sales_train = sales_train[sales_train['year'] != 2014]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [194]:
# Якутск Орджоникидзе, 56
sales_train.loc[sales_train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales_train.loc[sales_train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales_train.loc[sales_train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [100]:
sums = sales_train.groupby('item_id')['item_cnt_day'].sum().reset_index().rename(columns={"item_cnt_day":"item_total_sales"}).sort_values(by='item_total_sales')

ids_reject = sums[(sums['item_total_sales'] > 0) & (sums['item_total_sales'] < 1000)]['item_id'].unique()

#sums.groupby(pd.cut(sums["item_total_sales"], np.arange(0, 16000, 1000))).count()#.cumsum()


In [76]:
val_ids = training[training['date_block_num'] == 33]['item_id'].unique()
len_val = len(val_ids)
reject_for_val= val_ids[0:int(len_val/2)]

In [7]:
def get_number_of_days_since_start(day,month, year):
    days = 0
    if year == 2015:
        days = 365
    def is_even(num):
        return num % 2 == 0
    half_of_month = int(month/2)
    even = (30*half_of_month) + (31*half_of_month)
    if is_even(month):
        days = days + even - 30 - day
    else:
        days = days + even + day
    return days

sales_train['item_days_since_start'] = pd.to_numeric(sales_train.apply(lambda row: get_number_of_days_since_start(row['day'],row['month'], row['year']),axis=1), downcast='unsigned') 

In [77]:
train_block_2015 = sales_train[sales_train['year'] == 2015]['date_block_num'].unique()

In [195]:
train_item_ids = sales_train['item_id'].unique()
#train_item_ids = np.setdiff1d(train_item_ids, np.append(ids_reject,reject_for_val))
train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))

In [196]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    len_item_ids = len(sales.item_id.unique())
    sales_item_ids = sales.item_id.unique()#[0:int(len_item_ids/2)]
    len_shop_ids = len(sales.shop_id.unique())
    sales_shop_ids = sales.shop_id.unique()#[0:int(len_shop_ids/2)]
    dbn_combos = list(product(sales_shop_ids, sales_item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [197]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['item_cnt_block'] = training['item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [198]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')


In [199]:
training["shop_cat"] = training["shop_id"].astype(str) + "_" + training["item_category_id"].astype(str)

In [200]:
training["shop_item"] = training["shop_id"].astype(str) + "_" + training["item_id"].astype(str)

In [201]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
columns = ["item_id", "shop_id", "item_category_id", "month", "shop_cat", "shop_item", "date_block_num"]
#columns = ["item_id", "shop_id", "item_category_id", "month",  "date_block_num"]



y_train = training["item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    for column in columns:
        means = training.iloc[in_fold_index].groupby(column)['item_cnt_block'].mean()
            #x_validation[column + "_mean_target"] = means\
        name = column + '_mean_encoding'
        training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index][column].map(means)
    i+=1

fold 1
fold 2
fold 3
fold 4
fold 5


In [202]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
#columns = ["item_id", "shop_id", "item_category_id", "month", "shop_cat", "shop_item", "date_block_num"]



y_train = training["item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    #print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    total = training.iloc[in_fold_index].drop_duplicates(["item_id", "date_block_num"])['item_cnt_block'].sum()
    #print(total)
    fold_sum = training.iloc[in_fold_index].drop_duplicates(["item_id", "date_block_num"]).groupby("item_id")['item_cnt_block'].sum() *100 / total
    #print(fold_sum)
        #x_validation[column + "_mean_target"] = means\
    name = 'item_share_mean_encoding'
    training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index]["item_id"].map(fold_sum)
    i+=1

In [203]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
#columns = ["item_id", "shop_id", "item_category_id", "month", "shop_cat", "shop_item", "date_block_num"]



y_train = training["item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    total = training.iloc[in_fold_index]['item_cnt_block'].sum()
    print(total)
    #print(total)
    fold_sum = training.iloc[in_fold_index].groupby("shop_id")['item_cnt_block'].sum() *100 / total
    #print(fold_sum)
        #x_validation[column + "_mean_target"] = means\
    name = 'shop_share_mean_encoding'
    training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index]["shop_id"].map(fold_sum)
    i+=1

fold 1
1477590
fold 2
1478806
fold 3
1479154
fold 4
1480753
fold 5
1478441


In [204]:
training['shares_comp_1'] = training['shop_share_mean_encoding'] * training['item_share_mean_encoding']

In [205]:
sales_train["item_price_block_mean"] = sales_train.groupby(["item_id","date_block_num"])["item_price"].transform(np.mean)
sales_train["item_price_block_mean_max"] = sales_train.groupby("item_id")["item_price_block_mean"].transform(np.max)

training = training.merge(sales_train.drop_duplicates(["item_id", "date_block_num"])[["item_id", "date_block_num", "item_price_block_mean"]], how="left", on=["item_id", "date_block_num"])
training = training.merge(sales_train.drop_duplicates(["item_id"])[["item_id", "item_price_block_mean_max"]], how="left", on=["item_id"])

In [206]:
training["price_block_var_max"] = (training["item_price_block_mean_max"] - training["item_price_block_mean"]) / training["item_price_block_mean_max"]

In [207]:
def add_block_units_mean(df, cols, name):
    print(name)
    name_units = name + '_units'
    name_mean = name + '_mean'
    
    try:
        df.drop(columns=[name_units, name_mean],inplace=True)
    except:
        pass

    
    block_units = df.groupby(cols,as_index=False)['item_cnt_block'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_block':name_units})
    df = df.merge(block_units, on=cols, how='left')
    df[name_units].fillna(0,inplace=True)
    df[name_units] = pd.to_numeric(df[name_units].astype(int),downcast='unsigned')
    del block_units
    
    block_means = df.groupby(cols,as_index=False)['item_cnt_block'].mean()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_block':name_mean})
    df = df.merge(block_means, on=cols, how='left')
    df[name_mean].fillna(0,inplace=True)
    df[name_mean] = pd.to_numeric(df[name_mean],downcast='float')
    del block_means
    
    gc.collect()
    return df


training = add_block_units_mean(training, ['item_id','date_block_num'], 'item_block')
training = add_block_units_mean(training, ['shop_id','date_block_num'], 'shop_block')
training = add_block_units_mean(training, ['item_category_id','date_block_num'], 'cat_block')
training = add_block_units_mean(training, ['shop_id', 'item_category_id','date_block_num'], 'shop_cat_block')
training = add_block_units_mean(training, ['shop_id', 'item_id','date_block_num'], 'shop_item_block')

item_block
shop_block
cat_block
shop_cat_block
shop_item_block


In [208]:
def add_rolls(df, cols, name, rolls = [2]):
    for roll in rolls:
        print(name, roll)
        roll_name = name+"_rolling_" + str(roll)
        roll_name_tmp = roll_name + "_tmp"
        
        try:
            df.drop(columns=[roll_name],inplace=True)
        except:
            pass       

    
        block_units_rolling_temp = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].rolling(roll,min_periods=2).mean().reset_index()\
            .rename(columns={name:roll_name_tmp})\
            [cols+[roll_name_tmp]]
        
    
        df = df.merge(block_units_rolling_temp, on=cols, how='left')
        #print(df.columns.values)
        del block_units_rolling_temp
        gc.collect()
        

        block_units_rolling = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [roll_name_tmp].shift(1)\
            .rename(columns={roll_name_tmp:roll_name}).reset_index()

        df = df.merge(block_units_rolling, on=cols, how='left')
        df[roll_name].fillna(0,inplace=True)
        df[roll_name] = pd.to_numeric(df[roll_name], downcast='float')
        df.drop(columns=[roll_name_tmp], inplace=True)
        del block_units_rolling
        gc.collect()
    
    return df
    

#training = add_rolls(training, ['item_id','date_block_num'], 'item_block_units')
training = add_rolls(training, ['item_id','date_block_num'], 'item_block_mean')
#training = add_rolls(training, ['shop_id','date_block_num'], 'shop_block_units')
training = add_rolls(training, ['shop_id','date_block_num'], 'shop_block_mean')
#training = add_rolls(training, ['item_category_id','date_block_num'], 'cat_block_units')
training = add_rolls(training, ['item_category_id','date_block_num'], 'cat_block_mean')
#training = add_rolls(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')
training = add_rolls(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
#training = add_rolls(training, ['shop_id','item_id','date_block_num'], 'shop_item')

item_block_mean 2
shop_block_mean 2
cat_block_mean 2
shop_cat_block_mean 2


In [26]:
training = add_rolls(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

shop_item_block_mean 3


In [209]:
def add_lags(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()

        df = df.merge(result, on=cols, how='left')
        df[lag_name].fillna(0,inplace=True)
        if "mean" in name or "var" in name:
            df[lag_name] = pd.to_numeric(df[lag_name], downcast='float')
        else:
            df[lag_name] = pd.to_numeric(df[lag_name].astype(int), downcast='unsigned')
        del result
        gc.collect()
    
    return df
                                         

                                        
#training = add_lags(training, ['item_id','date_block_num'], 'item_block_units')
training = add_lags(training, ['item_id','date_block_num'], 'item_block_mean')
#training = add_lags(training, ['shop_id','date_block_num'], 'shop_block_units')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_block_mean')
#training = add_lags(training, ['item_category_id','date_block_num'], 'cat_block_units')
training = add_lags(training, ['item_category_id','date_block_num'], 'cat_block_mean')
#training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
#training = add_lags(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')
#training = add_lags(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')
training = add_lags(training, ['item_id','date_block_num'], 'price_block_var_max')

item_block_mean 1
shop_block_mean 1
cat_block_mean 1
shop_cat_block_mean 1
price_block_var_max 1


In [210]:
first_day = sales_train.groupby('item_id')['item_days_since_start'].min()
training['first_day'] = training['item_id'].map(first_day)

KeyError: 'Column not found: item_days_since_start'

In [211]:
training.fillna(0, inplace=True)

In [ ]:
training = training.sample(frac=1).reset_index(drop=True)


In [212]:
len(training)

6425094

In [163]:
cutoff = 50000
training2 = training.loc[0:cutoff,:]

In [213]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'item_cnt_block',
       'item_category_id', 'month', 'shop_cat', 'shop_item',
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding', 'month_mean_encoding',
       'shop_cat_mean_encoding', 'shop_item_mean_encoding',
       'date_block_num_mean_encoding', 'item_share_mean_encoding',
       'shop_share_mean_encoding', 'shares_comp_1',
       'item_price_block_mean', 'item_price_block_mean_max',
       'price_block_var_max', 'item_block_units', 'item_block_mean',
       'shop_block_units', 'shop_block_mean', 'cat_block_units',
       'cat_block_mean', 'shop_cat_block_units', 'shop_cat_block_mean',
       'shop_item_block_units', 'shop_item_block_mean',
       'item_block_mean_rolling_2', 'shop_block_mean_rolling_2',
       'cat_block_mean_rolling_2', 'shop_cat_block_mean_rolling_2',
       'item_block_mean_lag_1', 'shop_block_mean_lag_1',
       'cat_block_mean_lag_1', 'shop_cat_block_mean_lag_1',
 

In [146]:
len(training2)

100001

In [214]:
gc.collect()

ZEROS_KEEP=0.25


#x_train = training[(training['date_block_num'] < 33) & (training['val_ignore'] == False)]
x_train = training[(training['date_block_num'] < 33)]
y_train = x_train['item_cnt_block']





x_val = training[training['date_block_num'] == 33]
y_val = x_val['item_cnt_block']

pos_val_len = len(y_val[y_val != 0])
print("pos_val_len", pos_val_len)

zeros_keep_indices_val = y_val[y_val == 0].sample(int(pos_val_len/ZEROS_KEEP)).index
print("zeros_keep_indices_val", len(zeros_keep_indices_val))
non_zeros_val_indices = y_val[y_val != 0].index
print("non_zeros_val_indices", len(non_zeros_val_indices))

val_indices = np.append(np.array(zeros_keep_indices_val), np.array(non_zeros_val_indices))

#y_val = y_val.loc[val_indices]
#x_val = x_val.loc[val_indices]



pos_val_len 31471
zeros_keep_indices_val 125884
non_zeros_val_indices 31471


In [124]:
training.sample(1)

,item_id,shop_id,date_block_num,item_cnt_block,item_category_id,month,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding,month_mean_encoding,...,shop_cat_block_units,shop_cat_block_mean,shop_item_block_units,shop_item_block_mean,item_block_mean_lag_1,shop_block_mean_lag_1,price_block_var_max_lag_1,item_block_mean_rolling_2,shop_block_mean_rolling_2,cat_block_mean_rolling_2
5866732,15552,2,25,0,37,2,0.381773,0.142761,0.163802,0.293181,...,19,0.028274,0,0,0.4,0.153934,0.111392,0.43,0.209064,0.209181


In [119]:
training2.fillna(0,inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [265]:
features = [
      
       'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding', 'month_mean_encoding',
       'shop_cat_mean_encoding', 'shop_item_mean_encoding',
       'date_block_num_mean_encoding', 'item_share_mean_encoding',
       'shop_share_mean_encoding', 'shares_comp_1',
     'item_price_block_mean_max',
       'price_block_var_max',
       'item_block_mean_rolling_2', 'shop_block_mean_rolling_2',
       'cat_block_mean_rolling_2', 'shop_cat_block_mean_rolling_2',
       'item_block_mean_lag_1', 'shop_block_mean_lag_1',
       'cat_block_mean_lag_1', 'shop_cat_block_mean_lag_1',
       'price_block_var_max_lag_1'
]


gc.collect()
lgtrain = lgbm.Dataset(x_train[features], label=y_train)
lgval = lgbm.Dataset(x_val[features], label=y_val)



#[0.00542047893814942, 29, 24, 0.39949465609514856, 1, 0.67943500, 10]
params = {
        "num_threads": 16,
        #"device": "gpu",
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        #"max_bin": 10,#default 255
        #"num_leaves": 5, #default 31
        #"bagging_fraction": 0.2,
        #"bagging_freq": 1,
        "min_data_in_leaf": 10000,#default 20
        "feature_fraction": 0.5,
        #"lambda_l2": 3,
        #"max_depth": 4,
        #"min_gain_to_split": 10,
        "learning_rate" : 0.01,
        #"histogram_pool_size": 1000,
        #"categorical_column": [0,1,2,3,4]
}

evals_result = {}
lg_model = lgbm.train(params, lgtrain, 20000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=1, 
                      verbose_eval=10, 
                      evals_result=evals_result)

scores = {}
for i,score in enumerate(lg_model.feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

Training until validation scores don't improve for 1 rounds.
[10]	valid_0's rmse: 1.10117
[20]	valid_0's rmse: 1.07551
[30]	valid_0's rmse: 1.05245
[40]	valid_0's rmse: 1.03116
[50]	valid_0's rmse: 1.01453
[60]	valid_0's rmse: 0.998785
[70]	valid_0's rmse: 0.988179
[80]	valid_0's rmse: 0.978638
[90]	valid_0's rmse: 0.96804
[100]	valid_0's rmse: 0.961759
[110]	valid_0's rmse: 0.955011
[120]	valid_0's rmse: 0.950594
[130]	valid_0's rmse: 0.946367
[140]	valid_0's rmse: 0.942965
[150]	valid_0's rmse: 0.94003
Early stopping, best iteration is:
[154]	valid_0's rmse: 0.938643


[('item_block_mean_lag_1', 801),
 ('shop_item_mean_encoding', 487),
 ('item_block_mean_rolling_2', 399),
 ('item_id_mean_encoding', 374),
 ('shop_cat_mean_encoding', 316),
 ('price_block_var_max', 287),
 ('shop_id_mean_encoding', 277),
 ('shop_cat_block_mean_lag_1', 260),
 ('shares_comp_1', 199),
 ('date_block_num_mean_encoding', 189),
 ('shop_cat_block_mean_rolling_2', 160),
 ('shop_share_mean_encoding', 132),
 ('cat_block_mean_rolling_2', 126),
 ('item_category_id_mean_encoding', 114),
 ('shop_block_mean_lag_1', 111),
 ('item_share_mean_encoding', 90),
 ('month_mean_encoding', 90),
 ('shop_block_mean_rolling_2', 68),
 ('cat_block_mean_lag_1', 58),
 ('item_price_block_mean_max', 45),
 ('price_block_var_max_lag_1', 37)]

In [96]:
len(training)

614539

In [265]:
cb_model = CatBoostRegressor(iterations=1000,
                             #learning_rate=0.05,
                             eval_metric='RMSE',
                             task_type = "GPU",
                             use_best_model=True,
                             od_type = "Iter",
                             od_wait = 1,
                             bagging_temperature = 30,
                             cat_features=[0],
                             random_seed = 42)

#drops = ['subcategory','area']
#x_train = x_train.drop(columns=drops)
#x_val = x_val.drop(columns=drops)


cb_model.fit(x_train[features], y_train, #cat_features=categorical_features_indices,
             eval_set=(x_val[features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

scores = {}
for i,score in enumerate(cb_model.get_feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

0:	learn: 1.2142134	test: 1.3007728	best: 1.3007728 (0)	total: 208ms	remaining: 3m 27s
1:	learn: 1.2048024	test: 1.2913237	best: 1.2913237 (1)	total: 422ms	remaining: 3m 30s
2:	learn: 1.1935868	test: 1.2809283	best: 1.2809283 (2)	total: 612ms	remaining: 3m 23s
3:	learn: 1.1839645	test: 1.2724598	best: 1.2724598 (3)	total: 836ms	remaining: 3m 28s
4:	learn: 1.1767478	test: 1.2645890	best: 1.2645890 (4)	total: 1.01s	remaining: 3m 21s
5:	learn: 1.1705086	test: 1.2577772	best: 1.2577772 (5)	total: 1.22s	remaining: 3m 21s
6:	learn: 1.1622471	test: 1.2508170	best: 1.2508170 (6)	total: 1.4s	remaining: 3m 18s
7:	learn: 1.1528430	test: 1.2422691	best: 1.2422691 (7)	total: 1.62s	remaining: 3m 21s
8:	learn: 1.1469227	test: 1.2371112	best: 1.2371112 (8)	total: 1.83s	remaining: 3m 21s
9:	learn: 1.1379664	test: 1.2292195	best: 1.2292195 (9)	total: 2.03s	remaining: 3m 20s
10:	learn: 1.1309602	test: 1.2228338	best: 1.2228338 (10)	total: 2.22s	remaining: 3m 19s
11:	learn: 1.1239430	test: 1.2168717	best:

93:	learn: 0.9618758	test: 1.0653203	best: 1.0653203 (93)	total: 20.4s	remaining: 3m 16s
94:	learn: 0.9603815	test: 1.0641604	best: 1.0641604 (94)	total: 20.6s	remaining: 3m 15s
95:	learn: 0.9599992	test: 1.0639171	best: 1.0639171 (95)	total: 20.8s	remaining: 3m 15s
96:	learn: 0.9598076	test: 1.0637045	best: 1.0637045 (96)	total: 21s	remaining: 3m 15s
97:	learn: 0.9596032	test: 1.0636136	best: 1.0636136 (97)	total: 21.3s	remaining: 3m 16s
98:	learn: 0.9588431	test: 1.0626051	best: 1.0626051 (98)	total: 21.5s	remaining: 3m 16s
99:	learn: 0.9584068	test: 1.0621874	best: 1.0621874 (99)	total: 21.7s	remaining: 3m 15s
100:	learn: 0.9581137	test: 1.0617128	best: 1.0617128 (100)	total: 21.9s	remaining: 3m 15s
101:	learn: 0.9575106	test: 1.0612601	best: 1.0612601 (101)	total: 22.2s	remaining: 3m 15s
102:	learn: 0.9569574	test: 1.0608835	best: 1.0608835 (102)	total: 22.5s	remaining: 3m 15s
103:	learn: 0.9559245	test: 1.0600643	best: 1.0600643 (103)	total: 22.7s	remaining: 3m 15s
104:	learn: 0.9

185:	learn: 0.9294323	test: 1.0330034	best: 1.0330034 (185)	total: 40.8s	remaining: 2m 58s
186:	learn: 0.9286348	test: 1.0321979	best: 1.0321979 (186)	total: 41s	remaining: 2m 58s
187:	learn: 0.9282590	test: 1.0316546	best: 1.0316546 (187)	total: 41.2s	remaining: 2m 57s
188:	learn: 0.9281398	test: 1.0316287	best: 1.0316287 (188)	total: 41.4s	remaining: 2m 57s
189:	learn: 0.9281088	test: 1.0315856	best: 1.0315856 (189)	total: 41.6s	remaining: 2m 57s
190:	learn: 0.9279867	test: 1.0315170	best: 1.0315170 (190)	total: 41.9s	remaining: 2m 57s
191:	learn: 0.9279078	test: 1.0314832	best: 1.0314832 (191)	total: 42.1s	remaining: 2m 57s
192:	learn: 0.9271230	test: 1.0306199	best: 1.0306199 (192)	total: 42.3s	remaining: 2m 56s
193:	learn: 0.9269463	test: 1.0304205	best: 1.0304205 (193)	total: 42.5s	remaining: 2m 56s
194:	learn: 0.9268561	test: 1.0302560	best: 1.0302560 (194)	total: 42.9s	remaining: 2m 56s
195:	learn: 0.9266286	test: 1.0300495	best: 1.0300495 (195)	total: 43.1s	remaining: 2m 56s
1

[('item_block_mean_lag_1', 24.26178950360488),
 ('shop_item_block_mean_lag_1', 20.302278737207473),
 ('shop_item_block_mean_rolling_3', 11.060282556272488),
 ('shop_cat_block_mean_lag_1', 9.487496404150535),
 ('item_block_mean_rolling_3', 7.479725163588699),
 ('shop_cat_block_mean_rolling_3', 6.068712510620433),
 ('shop_item_share_of_shop_units_mean', 5.536159204862697),
 ('item_category_id', 5.138615825079335),
 ('cat_me_real', 4.480760033674159),
 ('shop_block_mean_lag_1', 3.542334724861135),
 ('shop_block_mean_rolling_3', 2.641845336078146)]

In [48]:
features = [item[0] for item in scores.items() if item[1] > 2000]

In [284]:
test            = pd.read_csv('test.csv.gz')
test = test.set_index('item_id').join(items.set_index('item_id'))
test.reset_index(inplace=True)
test['month'] = 11

In [285]:
item_features = [ 
    #'shop_item_share_of_shop_units_mean'
       'item_id_mean_encoding','price_block_var_max_lag_1','item_share_mean_encoding',"item_price_block_mean_max",'price_block_var_max'
                ]

merge_col = ['item_id']
cols=item_features+merge_col

test = test.merge(training.drop_duplicates('item_id')[cols], on=merge_col, how='left')

In [286]:
shop_features = [
        #'shop_me'
           'shop_id_mean_encoding','shop_share_mean_encoding'

]

merge_col = ['shop_id']
cols=shop_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [287]:
cat_features = [
        'item_category_id_mean_encoding','cat_block_mean_lag_1'
]

merge_col = ['item_category_id']
cols=cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [288]:
month_features = [
        'month_mean_encoding'
]

merge_col = ['month']
cols=month_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [289]:
shop_cat_features = [
        'shop_cat_mean_encoding','shop_cat_block_mean_rolling_2'
]

merge_col = ['shop_id', 'item_category_id']
cols=shop_cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [290]:
shop_item_features = [
        'shares_comp_1','shop_item_mean_encoding'
]

merge_col = ['shop_id', 'item_id']
cols=shop_item_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [291]:
test["date_block_num_mean_encoding"] = training[training["date_block_num"] == 33]["date_block_num_mean_encoding"].mean()

In [292]:
def add_rolls_test(df, cols, name, rolls = [2]):
    for roll in rolls:
        print(name, roll)
        roll_name = name+"_rolling_" + str(roll)
        roll_name_tmp = roll_name + "_tmp"
        
        try:
            df.drop(columns=[roll_name],inplace=True)
        except:
            pass       

    
        block_units_rolling_temp = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].rolling(roll,min_periods=2).mean().reset_index()\
            .rename(columns={name:roll_name})\
            [cols+[roll_name]]
        
        print([cols[0:len(cols)-1]+[roll_name]])
        thirty_three = block_units_rolling_temp[block_units_rolling_temp['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1]+[roll_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')
    

        del block_units_rolling_temp
        gc.collect()
        

    
    return df
    

test = add_rolls_test(test, ['item_id','date_block_num'], 'item_block_mean')
test = add_rolls_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
test = add_rolls_test(test, ['item_category_id','date_block_num'], 'cat_block_mean')



item_block_mean 2
[['item_id', 'item_block_mean_rolling_2']]
shop_block_mean 2
[['shop_id', 'shop_block_mean_rolling_2']]
cat_block_mean 2
[['item_category_id', 'cat_block_mean_rolling_2']]


In [179]:
test = add_rolls_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

shop_item_block_mean 2


KeyboardInterrupt: 

In [293]:
def add_lags_test(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()
        
        thirty_three = result[result['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1] + [lag_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')

        gc.collect()
    
    return df
                                         

                                        
test = add_lags_test(test, ['item_id','date_block_num'], 'item_block_mean')
test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
test = add_lags_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')


item_block_mean 1
shop_block_mean 1
shop_cat_block_mean 1


In [236]:
test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

shop_item_block_mean 1


In [182]:
test.sample(10)

,item_id,ID,shop_id,item_category_id,month,shop_id_mean_encoding_x,shop_share_mean_encoding_x,item_id_mean_encoding,price_block_var_max_lag_1,item_share_mean_encoding,shop_id_mean_encoding_y,shop_share_mean_encoding_y,item_category_id_mean_encoding,month_mean_encoding,shares_comp_1,date_block_num_mean_encoding,item_block_mean_rolling_2,item_block_mean_lag_1,shop_block_mean_lag_1,shop_cat_block_mean_lag_1
5502,800,4596,5,54,11,0.194066,1.424157,0.036388,0.000000,0.000000,0.194066,1.424157,0.152706,0.309452,0.000000,0.258565,0.000000,0.000000,0.205900,0.000000
214077,22164,16642,3,37,11,0.128040,0.937885,1.138264,0.000000,0.000000,0.128040,0.937885,0.164101,0.309452,0.004883,0.258565,0.275106,0.209302,0.132547,0.038123
136077,13802,201822,44,40,11,0.169537,1.241372,1.354839,0.010802,0.005190,0.169537,1.241372,0.238639,0.309452,0.025980,0.258565,0.378700,0.325581,0.149263,0.123883
186844,19051,145495,58,55,11,0.348356,2.565894,0.729412,0.077042,0.005186,0.348356,2.565894,0.224971,0.309452,0.013581,0.258565,0.172040,0.139535,0.255457,0.172360
91288,9494,114625,49,40,11,0.122322,0.893668,0.087819,0.232558,0.000000,0.122322,0.893668,0.238639,0.309452,0.000000,0.258565,0.057082,0.023256,0.111898,0.047254
46149,4676,171818,37,58,11,0.164947,1.213178,0.029412,0.000000,0.000000,0.164947,1.213178,0.043990,0.309452,0.000000,0.258565,0.000000,0.000000,0.184267,0.048544
78256,8113,52207,26,55,11,0.242860,1.768919,0.184615,0.000000,0.000000,0.242860,1.768919,0.224971,0.309452,0.000000,0.258565,0.170719,0.023256,0.238348,0.094720
39506,4055,132713,48,20,11,0.204023,1.248278,0.583333,0.000000,0.000000,0.204023,1.248278,1.502332,0.309452,0.000000,0.258565,0.000000,0.000000,0.214159,0.973214
119926,12715,82765,16,40,11,0.214748,1.580528,0.273038,0.052632,0.005190,0.214748,1.580528,0.238639,0.309452,0.008263,0.258565,0.000000,0.000000,0.184071,0.122605
139520,14217,197084,41,57,11,0.174963,1.276580,0.147410,0.063067,0.000000,0.174963,1.276580,0.094451,0.309452,0.000000,0.258565,0.000000,0.000000,0.129990,0.000000


In [116]:
cb_preds = cb_model.predict(test[features])
cb_preds.clip(0,20,out=cb_preds)

NameError: name 'cb_model' is not defined

In [271]:
print(np.mean(cb_preds))
print(np.max(cb_preds))

0.31713491408940697
11.488168775656959


In [294]:
test.fillna(0, inplace=True)
lg_preds = lg_model.predict(test[features])
lg_preds.clip(0,20,out=lg_preds)

array([ 0.16262253,  0.12507458,  0.20694521, ...,  0.29682143,
        0.35680383,  0.28541894])

In [295]:
print(np.mean(lg_preds))
print(np.max(lg_preds))

0.299653153412
8.84551683


In [294]:
print(np.mean(cb_preds))
print(np.max(cb_preds))

NameError: name 'cb_preds' is not defined

In [253]:
lg_preds[0:100]

array([ 0.50527568,  0.49098723,  0.57521519,  0.49098723,  0.49098723,
        0.5280722 ,  0.49098723,  0.50635039,  0.57521519,  0.57521519,
        0.50635039,  0.57521519,  0.50635039,  0.50635039,  0.57521519,
        0.5280722 ,  0.50527568,  0.50635039,  0.49098723,  0.57521519,
        0.57521519,  0.50527568,  0.49098723,  0.50635039,  0.50527568,
        0.57521519,  0.5034492 ,  0.57521519,  0.57521519,  0.49388843,
        0.50635039,  0.57521519,  0.49098723,  0.49098723,  0.57521519,
        0.50635039,  0.49098723,  0.57521519,  0.49098723,  0.49098723,
        0.49098723,  0.49098723,  0.53578509,  0.52149664,  0.61880098,
        0.52149664,  0.52149664,  0.56315929,  0.52149664,  0.53578509,
        0.61880098,  0.61880098,  0.53578509,  0.61880098,  0.53578509,
        0.53578509,  0.61880098,  0.56315929,  0.53578509,  0.53578509,
        0.52149664,  0.61880098,  0.61880098,  0.53578509,  0.52149664,
        0.53578509,  0.53578509,  0.61880098,  0.53288389,  0.61

In [296]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = lg_preds

submission.to_csv('submission.csv', index=False)

In [351]:
training.sample(10)

,item_id,shop_id,date_block_num,item_cnt_block,item_category_id,month,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding,month_mean_encoding,date_block_num_mean_encoding,first_day,item_block_units,item_block_mean,shop_block_units,shop_block_mean,cat_block_units,cat_block_mean,shop_cat_block_units,shop_cat_block_mean,shop_item_block_units,shop_item_block_mean,item_block_mean_rolling_2,item_block_mean_rolling_3,item_block_mean_rolling_4,item_block_mean_rolling_6,shop_block_mean_rolling_2,shop_block_mean_rolling_3,shop_block_mean_rolling_4,shop_block_mean_rolling_6,cat_block_mean_rolling_2,cat_block_mean_rolling_3,cat_block_mean_rolling_4,cat_block_mean_rolling_6
1664544,5983,58,27,0,30,4,0.039655,0.347876,0.991726,0.250437,0.242989,14,1,0.021277,1629,0.297480,7826,0.867243,209,1.088542,0,0,0.061739,0.061159,0.050870,0.043782,0.377824,0.382034,0.457419,0.414623,0.862122,0.904061,0.977607,1.024919
2885211,10424,5,13,0,37,2,0.084084,0.194460,0.163523,0.293575,0.307729,8,1,0.021739,1191,0.166947,6026,0.151270,59,0.068129,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5083670,17482,48,30,0,37,7,0.304575,0.206393,0.163523,0.256729,0.260998,1,5,0.116279,1094,0.205523,3291,0.156834,40,0.081967,0,0,0.264006,0.296572,0.303950,0.329158,0.191532,0.186673,0.187645,0.192634,0.157556,0.168180,0.167541,0.167832
2419986,8708,22,25,0,37,2,0.040000,0.214979,0.163523,0.293575,0.277288,68,1,0.021277,1221,0.201718,4830,0.152926,72,0.107143,0,0,0.059412,0.046411,0.045012,0.043755,0.260791,0.255428,0.247499,0.236772,0.209181,0.191768,0.177641,0.163198
3798403,13436,16,22,1,11,11,0.061404,0.218308,0.260647,0.311852,0.311852,1,2,0.040000,1562,0.247074,92,0.184000,3,0.300000,1,1,0.069231,0.072298,0.059223,0.056489,0.211654,0.218686,0.213326,0.208407,0.140140,0.159974,0.162481,0.191473
2276070,8248,31,15,1,40,4,0.054913,1.119358,0.239173,0.250437,0.256209,6,2,0.040816,7464,1.101860,17333,0.224309,1990,1.261890,1,1,0.063859,0.078804,0.078804,0.078804,1.145991,1.147343,1.147343,1.147343,0.246038,0.249882,0.249882,0.249882
1073356,3970,39,30,0,55,7,0.253191,0.122355,0.224271,0.257048,0.258174,244,7,0.162791,598,0.112343,6475,0.216664,63,0.090647,0,0,0.206395,0.222703,0.270288,0.266504,0.123662,0.120304,0.123071,0.123010,0.207113,0.209156,0.221755,0.225911
4027367,14220,10,22,0,57,11,0.085906,0.098287,0.094370,0.311019,0.311019,7,4,0.080000,717,0.113413,697,0.085521,0,0.000000,0,0,0.079020,0.079346,0.090122,0.107700,0.101840,0.100006,0.095387,0.094615,0.084507,0.092603,0.097591,0.100657
4113557,14434,7,14,0,40,3,0.103718,0.265655,0.238764,0.288169,0.299465,2,11,0.229167,1905,0.266732,20143,0.253104,289,0.174306,0,0,0.260870,0.260870,0.260870,0.260870,0.301063,0.301063,0.301063,0.301063,0.248271,0.248271,0.248271,0.248271
2458455,8820,55,15,0,65,4,0.178462,0.244115,0.653119,0.250581,0.255012,2,7,0.142857,998,0.147328,1838,0.721350,0,0.000000,0,0,0.338768,0.349034,0.349034,0.349034,0.133020,0.133310,0.133310,0.133310,0.849364,0.914532,0.914532,0.914532


In [169]:
sales_train.sample(10)

,item_id,date,date_block_num,shop_id,item_price,item_cnt_day,day,month,year,item_category_id
635476,6501,16.02.2015,25,42,390.64,1.0,16,2,2015,28
1271389,16056,21.01.2014,12,25,699.00,1.0,21,1,2014,64
1123093,14124,03.07.2015,30,41,599.00,1.0,3,7,2015,41
982533,11797,28.12.2014,23,31,149.00,1.0,28,12,2014,41
307649,3566,03.02.2014,13,27,407.36,1.0,3,2,2014,23
1363132,17164,14.08.2014,19,49,399.00,1.0,14,8,2014,40
1081679,13491,19.08.2014,19,52,14990.00,1.0,19,8,2014,11
1537875,20243,13.06.2015,29,16,199.00,1.0,13,6,2015,40
884686,10207,06.08.2015,31,49,1199.00,1.0,6,8,2015,30
542943,5673,03.09.2014,20,28,799.00,1.0,3,9,2014,3
